In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv("/work/Reviews-20231109-095328.csv")

In [ ]:
df['Text']

0         I have bought several of the Vitality canned d...
1         Product arrived labeled as Jumbo Salted Peanut...
2         This is a confection that has been around a fe...
3         If you are looking for the secret ingredient i...
4         Great taffy at a great price.  There was a wid...
                                ...                        
568449    Great for sesame chicken..this is a good if no...
568450    I'm disappointed with the flavor. The chocolat...
568451    These stars are small, so you can give 10-15 o...
568452    These are the BEST treats for training and rew...
568453    I am very satisfied ,product is as advertised,...
Name: Text, Length: 568454, dtype: object

In [ ]:
import nltk
import re
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

In [ ]:

def remove_html_tags(text):
  pattern=re.compile('<.*?>')
  return pattern.sub(r'',text)


def remove_url(text):
  pattern=re.compile(r'https?://\S+|WWW\.\S+')
  return pattern.sub(r'',text)


import string
string.punctuation
exclude=string.punctuation

def remove_punc(text):
  for char in exclude:
    text=text.replace(char,'')
  return text


def remove_stopwords(text):
  new_text=[]

  for word in text.split():
    if word in stopwords.words('english'):
      new_text.append('')
    else:
      new_text.append(word)

  x=new_text[:]
  new_text.clear()
  return " ".join(x)


def stem_words(text):
  return " ".join([ps.stem(word) for word in text.split()])


ps=PorterStemmer()
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:

tex=[]
for fea in df['Text']:
  fea=fea.lower()
  tex.append(fea)
df['Text']=tex
df['Text']

0         i have bought several of the vitality canned d...
1         product arrived labeled as jumbo salted peanut...
2         this is a confection that has been around a fe...
3         if you are looking for the secret ingredient i...
4         great taffy at a great price.  there was a wid...
                                ...                        
568449    great for sesame chicken..this is a good if no...
568450    i'm disappointed with the flavor. the chocolat...
568451    these stars are small, so you can give 10-15 o...
568452    these are the best treats for training and rew...
568453    i am very satisfied ,product is as advertised,...
Name: Text, Length: 568454, dtype: object

In [ ]:


     

df['Text']=df['Text'].apply(remove_html_tags)


In [ ]:
df['Text']=df['Text'].apply(remove_url)



In [ ]:
df['Text']=df['Text'].apply(remove_punc)

In [ ]:
 df['Text']=df['Text'].apply(stem_words)

In [0]:
df['Text']=df['Text'].apply(remove_stopwords)

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping


2023-11-10 06:28:06.148152: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-10 06:28:06.366360: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-11-10 06:28:06.366430: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-11-10 06:28:06.434174: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-10 06:28:09.393806: W tensorflow/stream_executor/platform/de

In [ ]:

from tensorflow.keras.layers import Dropout

In [ ]:
### Vocabulary size
voc_size=5000
     

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in df['Text']]
onehot_repr

[[512,
  1842,
  3113,
  4915,
  4789,
  2253,
  3860,
  1815,
  3340,
  2253,
  1188,
  1106,
  4344,
  2353,
  2713,
  4969,
  947,
  1489,
  1401,
  3390,
  1933,
  2253,
  947],
 [2253,
  2039,
  474,
  3729,
  2196,
  4156,
  1648,
  1531,
  1957,
  2435,
  4251,
  4746,
  1933,
  3775,
  1020,
  3700,
  1888,
  2710,
  2253,
  3729],
 [1933,
  1201,
  2620,
  3371,
  3580,
  495,
  4904,
  883,
  2673,
  3504,
  1933,
  1565,
  227,
  4979,
  988,
  1758,
  2515,
  3578,
  2807,
  58,
  988,
  3267,
  1433,
  2628,
  155,
  1852,
  4146,
  3336,
  1933,
  1172,
  3994,
  2238,
  512,
  2257,
  3691,
  2169,
  3884,
  2674,
  1933,
  3994,
  3742,
  4146,
  1113,
  1574,
  3371,
  4066,
  3884],
 [1188,
  680,
  2715,
  4267,
  3127,
  3860,
  3120,
  1933,
  979,
  4539,
  1178,
  4956,
  297,
  3775,
  1815,
  4372,
  369,
  719,
  1852,
  155,
  1462],
 [72,
  1739,
  72,
  380,
  3775,
  4976,
  943,
  1172,
  1739,
  4962,
  3775,
  155,
  795,
  1739,
  2809,
  1933,
  4361]

In [ ]:
sent_length=50
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)
     

[[   0    0    0 ... 1933 2253  947]
 [   0    0    0 ... 2710 2253 3729]
 [   0    0    0 ... 3371 4066 3884]
 ...
 [   0    0    0 ... 1106 4915 4789]
 [   0    0    0 ... 1909 1262 3994]
 [   0    0    0 ... 3832 3217  147]]


In [ ]:
## Creating model
embedding_vector_features=50 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(128,return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(64,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(32,return_sequences=True))
model.add(Flatten())
model.add(Dense(10,activation='softmax'))
model.add(Flatten())
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 50, 50)            250000    
                                                                 
 lstm_6 (LSTM)               (None, 50, 128)           91648     
                                                                 
 dropout_4 (Dropout)         (None, 50, 128)           0         
                                                                 
 lstm_7 (LSTM)               (None, 50, 64)            49408     
                                                                 
 dropout_5 (Dropout)         (None, 50, 64)            0         
                                                                 
 lstm_8 (LSTM)               (None, 50, 32)            12416     
                                                                 
 flatten_4 (Flatten)         (None, 1600)             

In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(df['Score'])
     

In [ ]:
X_final.shape,y_final.shape

((568454, 50), (568454,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33)

callback= EarlyStopping(monitor='val_loss', min_delta=0.00001, patience=20, verbose=1, mode='auto', baseline=None, restore_best_weights=False)

In [0]:
model.fit(X_train,y_train,validation_split=0.2,epochs=3500,batch_size=256,callbacks=callback)

Epoch 1/5
1191/1191 [==============================] - 1533s 1s/step - loss: 0.8774 - accuracy: 0.6865 - val_loss: 0.7925 - val_accuracy: 0.7106
Epoch 2/5
1191/1191 [==============================] - 1536s 1s/step - loss: 0.7678 - accuracy: 0.7168 - val_loss: 0.7579 - val_accuracy: 0.7195
Epoch 3/5
 514/1191 [===========>..................] - ETA: 13:23 - loss: 0.7273 - accuracy: 0.7306

In [ ]:
model.evaluate(X_test,y_test)
#Dear mentors ,these codes will run and give validation accuracy above 0.80 with no issues of underfitting and overfitting.Due to the memory and hardware issues,I couldnt perform this completely.Kindly check my codes,Its written in a proper manner

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4380d0cb-485b-4cb7-b5db-204bdf03d932' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>